In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


Bibliography 
@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}

Before starting the exploratary data analysis, I first want to see what the researchers where thinking as they made this dataset. From the paper, we see that it contains 50K reviews with no more thana 30 reviews per movie, 25K for training and 25K for testing with additional unlabelled data. They also mention the reviews are highly polar - for binary sentiment classification. One possible issue with the dataset could be how its inaccurate to real world feelings; there are cases where users misrepresent the reviews with stars - highly rated but its a parody comment for example. It doesn't capture sarcasm unfortunately and might take a sardonic review as positive.  

The model has higher accuracy compared to the listed models. We can start the EDA by first looking at what information we can extrapolate from the properties, tables, etc: 

We will start by first looking at columns and related properties to get an understading of what is being tracked: 

In [41]:
ds = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


As mentioned, the datset has two columns review & sentiment; they contain upto 50K object non-null values. Let's look at other features: 

In [42]:
ds.sentiment.value_counts() #contains only positive & negative values
sentiment_label=ds.sentiment.factorize() #map positive to 0, negative to 1


In [43]:
ds.describe() # dataset properties 

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [44]:
ds.head() #to get an idea of what the table looks like 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [45]:
ds.review[3] #sample review to see what it looks like

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

we see that its got the break HTML elements in the code - we will need to remove such elements. Let's take out by using replace with regex: 

In [46]:
ds['review'] = ds['review'].replace('<br />|[:\'"#{}_]','', regex=True)
# ds['review'] = ds['review'].str.replace('[^\w\s]','')
ds.review[1] #sample review to see result 

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only has got all the polari but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great masters of comedy and his life. The realism really comes home with the little things the fantasy of the guard which, rather than use the traditional dream techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwells murals decorating every surface) are terribly well done.'

Now we can split the data for training; the common approach is 80% for training and 20% for testing: 

In [47]:
shuffled = ds.sample(frac=1, random_state=1).reset_index()   #shuffle the values around for training
shuffled.head()

,index,review,sentiment
0,26247,With No Dead Heroes you get stupid lines like ...,negative
1,35067,I thought maybe... maybe this could be good. A...,negative
2,34590,An elite American military team which of cours...,negative
3,16668,Ridiculous horror film about a wealthy man (Jo...,negative
4,12196,"Well, if you are one of those Katanas film-nut...",positive


In [ ]:
# random split train and test data
index = ds.index
shuffled['random'] = np.random.randn(len(index))
train = shuffled[shuffled['random'] <= 0.8]
test = shuffled[shuffled['random_'] > 0.8]

In [69]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix,classification_report

In [55]:
X = np.array(shuffled.iloc[:,0].values)
Y = np.array(shuffled.sentiment.values)
CV = CountVectorizer(max_features = 1000)
X = CV.fit_transform(shuffled.review).toarray()
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


In [56]:
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=9)
print("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

Train shapes : X = (40000, 1000), y = (40000,)
Test shapes : X = (10000, 1000), y = (10000,)


In [59]:
sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(trainx, trainy)

SGDClassifier(random_state=42)

In [71]:
trained = sgd_clf.predict(testx)
print(classification_report(trained,testy))

              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      4925
    positive       0.87      0.85      0.86      5075

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

